In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
episodeDataPath = "/home/josh/GitHub/imdb-tv-ratings/all-episode-ratings.csv"
seriesDataPath = "/home/josh/GitHub/imdb-tv-ratings/top250list.csv"

episodeRatingData = pd.read_csv(episodeDataPath)

episodeRatingData.rename(columns={'Unnamed: 0': 'Episode Number'},
                        inplace=True)


episodeCounts = episodeRatingData['Code'].value_counts().to_frame()

episodeCounts.columns = ['Number of Episodes']

episodeRatingData = episodeRatingData.join(episodeCounts, on='Code')

episodeRatingData['Episode Number'] = episodeRatingData['Episode Number']+1

episodeRatingData['Prop Through Series'] = (episodeRatingData['Episode Number']/
                                           episodeRatingData['Number of Episodes'])

seriesData = pd.read_csv(seriesDataPath)

relevantSeriesData = seriesData[['Code', 'Title']]

In [3]:
seriesData

Code                Title     Code.1  Rating Rating Count  Rank
0    tt5491994      Planet Earth II  tt5491994     9.5       69,834     1
1    tt0185906     Band of Brothers  tt0185906     9.4      314,596     2
2    tt0944947      Game of Thrones  tt0944947     9.4    1,477,002     3
3    tt0795176         Planet Earth  tt0795176     9.4      146,143     4
4    tt0903747         Breaking Bad  tt0903747     9.4    1,185,584     5
..         ...                  ...        ...     ...          ...   ...
245  tt7767422        Sex Education  tt7767422     8.3       60,810   246
246  tt3671754          Kardes Payi  tt3671754     8.3       13,248   247
247  tt7078180    Violet Evergarden  tt7078180     8.3        5,058   248
248  tt2149175        The Americans  tt2149175     8.3       72,542   249
249  tt4156586  Permanent Roommates  tt4156586     8.3       18,829   250

[250 rows x 6 columns]

In [7]:
tvShowDataWithSeriesName = episodeRatingData.merge(relevantSeriesData, 
                                                   on='Code')

In [8]:
### The code for our version of the office is: tt0386676
tvShowDataWithSeriesName

Episode Number  Season  Episode  Rating       Code  Number of Episodes  \
0                   1       1        1     8.0  tt5491994                   6   
1                   2       1        2     7.3  tt5491994                   6   
2                   3       1        3     6.9  tt5491994                   6   
3                   4       1        4     6.8  tt5491994                   6   
4                   5       1        5     6.8  tt5491994                   6   
...               ...     ...      ...     ...        ...                 ...   
19909               9       2        4     7.1  tt4156586                  13   
19910              10       2        5     7.5  tt4156586                  13   
19911              11       2        6     7.7  tt4156586                  13   
19912              12       2        7     8.1  tt4156586                  13   
19913              13       2        8     8.8  tt4156586                  13   

       Prop Through Series                Title  
0                 0.166667      Planet Earth II  
1                 0.333333      Planet Earth II  
2                 0.500000      Planet Earth II  
3                 0.666667      Planet Earth II  
4                 0.833333      Planet Earth II  
...                    ...                  ...  
19909             0.692308  Permanent Roommates  
19910             0.769231  Permanent Roommates  
19911             0.846154  Permanent Roommates  
19912             0.923077  Permanent Roommates  
19913             1.000000  Permanent Roommates  

[19914 rows x 8 columns]

In [255]:
def televisionSeriesCondenser(tvSeries, numberOfEps):
    
    neededQuantiles = np.linspace(1, numberOfEps, numberOfEps)/numberOfEps
    
    episodesChosen = np.floor(
        np.quantile(tvSeries['Episode Number'], neededQuantiles))
    
    condensedShow = tvSeries.loc[np.isin(tvSeries['Episode Number'],
                                           episodesChosen)  , :]
    
    return(condensedShow)
    
    
    
    

In [256]:
relevantShows = tvShowDataWithSeriesName.loc[
    tvShowDataWithSeriesName['Number of Episodes'] >= 50, :]

In [257]:
americanOffice = relevantShows.loc[relevantShows['Code']=='tt0386676', :]

In [258]:
condensedOffice = televisionSeriesCondenser(tvSeries=americanOffice, numberOfEps=50)

In [259]:
condensedShows = relevantShows.groupby('Code').apply(
    lambda x: televisionSeriesCondenser(x, 51))

In [264]:
condensedShows = condensedShows.reset_index(level=0, drop=True)

In [274]:
condensedShows['Code'].value_counts().value_counts()

51    85
50     5
Name: Code, dtype: int64

In [292]:
onlyFirstFifty = condensedShows.groupby('Code').head(50).copy()

In [305]:
onlyFirstFifty.loc[:, 'main_character_left'] = (onlyFirstFifty['Title'] == 'The Office') & (
    onlyFirstFifty['Episode Number'] >138)

In [306]:
onlyFirstFifty['main_character_left'] = onlyFirstFifty['main_character_left'].astype('int32')

In [311]:
onlyFirstFifty.to_csv('tv rating data.csv', index=False)